In [2]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import glob
import pandas as pd

In [59]:
#importing data from txt
#all outputs from log should be on same line (i.e. every line starts with a time stamp)
dirc = os.getcwd()
data = pd.DataFrame()
filenames = glob.glob(dirc+'/*.txt')
positions = []
mm_order = []
size = []
timestamp = []
price = []
for file in filenames:
    f = open(file,"r")
    for line in f:
        #removes first few lines on text
        if re.search(r'(^\d*0)', line) == None:
            print(line)
            continue
        try:
            timestamp.append(int(re.search(r'(^\d*0)', line).group(1)))
        except:
            print('failed')
            
        if 'position: ' in line:
            positions.append(int(re.search(r'position: (-?\d*)', line).group(1)))
            if 'MM' in line:
                mm_order.append('MM')
                size.append(str(re.findall(r'(\d*.?\d*)x', line)[0])+'/'+str(re.findall(r'(\d*.?\d*)x', line)[1]))
                price.append(str(re.findall(r'x (\d*)', line)[0])+'/'+str(re.findall(r'x (\d*)', line)[1]))
            elif 'MISSED' in line:
                mm_order.append('MISSED')
                size.append(float(re.search(r'(\d*.?\d*)x', line).group(1)))
                price.append(float(re.search(r'x (\d*)', line).group(1)))
            else:
                mm_order.append(re.search(r'(SELL|BUY)', line).group(1))
                size.append(float(re.search(r'(\d*.?\d*)x', line).group(1)))
                price.append(float(re.search(r'x (\d*)', line).group(1)))
        else:
            positions.append(np.nan)
            mm_order.append(np.nan)
            size.append(np.nan)
            price.append(np.nan)

data['timestamp'] = timestamp
data['Position'] = positions
data['MM/order'] = mm_order
data['Size_available'] = size
data['Price'] = price
data2 = data.dropna()

print(data2)
#print(data)
#print(data.describe())

Sandbox logs:

START RequestId: d705ca66-ad61-4b63-8cdc-cbdb086c934d Version: $LATEST

Spread:  7 MM BUY PEARLS 5.0x 9999 position: 0 MM SELL PEARLS 5.0x 10004 position: 0

END RequestId: d705ca66-ad61-4b63-8cdc-cbdb086c934d

REPORT RequestId: d705ca66-ad61-4b63-8cdc-cbdb086c934d	Duration: 15.36 ms	Billed Duration: 16 ms	Memory Size: 128 MB	Max Memory Used: 65 MB	Init Duration: 543.33 ms	







Submission logs:

     timestamp  Position MM/order Size_available       Price
1          100      -1.0     SELL            1.0     10002.0
2          200      -1.0       MM        5.0/4.0  9997/10003
3          300      -2.0       MM        5.0/4.0  9996/10004
4          400      -2.0       MM        5.0/4.0  9999/10003
5          500      -2.0       MM        5.0/4.0  9999/10003
..         ...       ...      ...            ...         ...
995      99500       0.0      BUY            3.0      9998.0
996      99600       0.0       MM        5.0/5.0  9996/10004
997      99700       0.0       MM 

In [70]:
data2['Pos_T-1'] = data2['Position'].shift()
data2['Pos_T-1'][1] = 0
data2['Change'] = data2['Position'] - data2['Pos_T-1']
print(data2)

     timestamp  Position MM/order Size_available       Price  Pos_T-1  Change
1          100      -1.0     SELL            1.0     10002.0      0.0    -1.0
2          200      -1.0       MM        5.0/4.0  9997/10003     -1.0     0.0
3          300      -2.0       MM        5.0/4.0  9996/10004     -1.0    -1.0
4          400      -2.0       MM        5.0/4.0  9999/10003     -2.0     0.0
5          500      -2.0       MM        5.0/4.0  9999/10003     -2.0     0.0
..         ...       ...      ...            ...         ...      ...     ...
995      99500       0.0      BUY            3.0      9998.0     -3.0     3.0
996      99600       0.0       MM        5.0/5.0  9996/10004      0.0     0.0
997      99700       0.0       MM        5.0/5.0  9996/10004      0.0     0.0
998      99800       2.0       MM        4.0/5.0  9997/10001      0.0     2.0
999      99900       2.0       MM        4.0/5.0  9999/10003      2.0     0.0

[940 rows x 7 columns]


C:\Users\tommy\AppData\Local\Temp\ipykernel_7624\1323292632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Pos_T-1'] = data2['Position'].shift()
C:\Users\tommy\AppData\Local\Temp\ipykernel_7624\1323292632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Pos_T-1'][1] = 0
C:\Users\tommy\AppData\Local\Temp\ipykernel_7624\1323292632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o